# XGBoost Algorithm

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, accuracy_score, f1_score, confusion_matrix, classification_report,  roc_auc_score
import os
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from scipy import stats
import seaborn as sns
from itertools import product
import time
import json
from sklearn.model_selection import RandomizedSearchCV


In [13]:
import xgboost as xgb
from functions import *

## Load data

In [14]:
base_path = '/Users/leilapaolini/Documents/data'
full_path = os.path.join(base_path, 'data_y.npy')
data = np.load(full_path, allow_pickle=True)

print(f"Data type: {type(data)}")
print(f"Data shape: {data.shape if hasattr(data, 'shape') else 'N/A'}")

Data type: <class 'numpy.ndarray'>
Data shape: (9618522, 35)


In [15]:
df = pd.DataFrame(data, columns=['commune_1', 'commune_2', 'distance', 'year', 'pop_1', 'pop_2',
       'T_Mann', 'T_Frau', 'Etr_Total', 'Accidents dégâts matériels',
       'Accidents avec dommages corporels', 'Morts', 'GEM_FLAECH',
       'EINWOHNERZ', '0-25', '25-65', '65+', 'canton_code', 'unemployment',
       'gdp', 'T_Mann_2', 'T_Frau_2', 'Etr_Total_2',
       'Accidents dégâts matériels_2', 'Accidents avec dommages corporels_2',
       'Morts_2', 'GEM_FLAECH_2', 'EINWOHNERZ_2', '0-25_2', '25-65_2', '65+_2',
       'canton_code_2', 'unemployment_2', 'gdp_2', 'flow'])

df.head()



,commune_1,commune_2,distance,year,pop_1,pop_2,T_Mann,T_Frau,Etr_Total,Accidents dégâts matériels,...,Morts_2,GEM_FLAECH_2,EINWOHNERZ_2,0-25_2,25-65_2,65+_2,canton_code_2,unemployment_2,gdp_2,flow
0,1.0,2.0,3062.019741,2018.0,1982.0,12229.0,982.0,1000.0,135.0,3.0,...,0.0,1059.0,11900.0,3062.0,6981.0,2472.0,1.0,21793.250000,156883.07738,122.0
1,1.0,2.0,3062.019741,2020.0,1982.0,12229.0,994.0,1020.0,143.0,7.0,...,0.0,1059.0,12229.0,2951.0,7052.0,2551.0,1.0,26155.333333,149208.52863,79.0
2,1.0,3.0,4916.726674,2018.0,1982.0,5548.0,982.0,1000.0,135.0,3.0,...,0.0,743.0,5435.0,1532.0,3151.0,997.0,1.0,21793.250000,156883.07738,10.0
3,1.0,3.0,4916.726674,2020.0,1982.0,5548.0,994.0,1020.0,143.0,7.0,...,1.0,743.0,5548.0,1551.0,3107.0,1050.0,1.0,26155.333333,149208.52863,29.0
4,1.0,4.0,4951.312604,2018.0,1982.0,3701.0,982.0,1000.0,135.0,3.0,...,1.0,1360.0,3571.0,971.0,2072.0,739.0,1.0,21793.250000,156883.07738,17.0


In [16]:
n_total = len(df)
n_zero = (df['flow'] == 0).sum()

percent_zero = 100 * n_zero / n_total

print(f"Zero flows: {n_zero} / {n_total} ({percent_zero:.2f}%)")


Zero flows: 9492963 / 9618522 (98.69%)


## Data preparation

In [17]:
not_log_transform = ['commune_1', 'commune_2', 'Morts','Morts_2', 'year', 'canton_code', 'unemployment','gdp','canton_code_2', 'unemployment_2', 'gdp_2', 'flow']
to_log = [col for col in df.columns if col not in not_log_transform]

df = features_change(df, to_log, "log")



In [18]:
df = features_engineering(df)

Column for multiplied population added
Column for population ratio added
Column for difference between gdp of the communes added
Column for distance between the communes squared added
Column for gravitation added


notice that we do not normalise as xgboost does not need normalisation 

In [19]:
drop_features = ['commune_1', 'commune_2', 'flow']
X_train, y_train, X_val, y_val, X_test, y_test = build_train_test_val(df, 
                                                                      test_canton_ids=[3], 
                                                                      val_canton_ids=[19], 
                                                                      zero_drop_ratio=0, 
                                                                      random_state=37, 
                                                                      features=drop_features, 
                                                                      classify= True)

Splitting with canton [3] as test set:
Total flows: 9,618,522
Train size: 8,912,666 rows (92.7%)
Test size: 705,856 rows (7.3%)
Splitting with canton [19] as test set:
Total flows: 8,912,666
Train size: 7,227,626 rows (81.1%)
Test size: 1,685,040 rows (18.9%)
Dropping 0% of zeros in training data
Flow reclassified as 1: flow present, 0: no flow
Flow reclassified as 1: flow present, 0: no flow
Flow reclassified as 1: flow present, 0: no flow


In [20]:
X_val.head()

,distance,pop_1,pop_2,T_Mann,T_Frau,Etr_Total,Accidents dégâts matériels,Accidents avec dommages corporels,Morts,GEM_FLAECH,...,65+_2,canton_code_2,unemployment_2,gdp_2,pop_1_x_pop_2,pop_1/pop_2_ratio,gdp_minus_gdp_2,gravity_raw,year_2018.0,year_2020.0
2458,109.569788,7.591862,9.976087,6.889591,6.907755,4.905275,1.098616,0.693152,0.0,6.673298,...,8.385717,19.0,9734.916667,43687.13571,75.737075,0.761005,113195.94167,0.006309,1,0
2459,109.569788,7.591862,9.976087,6.901737,6.927558,4.962845,1.945912,1.609440,0.0,6.673298,...,8.392310,19.0,12802.583333,44108.19224,75.737075,0.761005,105100.33639,0.006309,0,1
2460,109.248387,7.591862,7.359468,6.889591,6.907755,4.905275,1.098616,0.693152,0.0,6.673298,...,6.063785,19.0,9734.916667,43687.13571,55.872061,1.031576,113195.94167,0.004681,1,0
2461,109.248387,7.591862,7.359468,6.901737,6.927558,4.962845,1.945912,1.609440,0.0,6.673298,...,6.142037,19.0,12802.583333,44108.19224,55.872061,1.031576,105100.33639,0.004681,0,1
2462,108.309143,7.591862,8.982938,6.889591,6.907755,4.905275,1.098616,0.693152,0.0,6.673298,...,7.239215,19.0,9734.916667,43687.13571,68.197221,0.845141,113195.94167,0.005813,1,0


In [21]:
# we do one-hot encoding for canton ids since they are categorical features 

X_train = pd.get_dummies(
    X_train,
    columns=["canton_code", "canton_code_2"],
    prefix=["canton_1", "canton_2"],
    drop_first=False
)

X_test = pd.get_dummies(
    X_test,
    columns=["canton_code", "canton_code_2"],
    prefix=["canton_1", "canton_2"],
    drop_first=False
)

X_val = pd.get_dummies(
    X_val,
    columns=["canton_code", "canton_code_2"],
    prefix=["canton_1", "canton_2"],
    drop_first=False
)

X_train.head()

,distance,pop_1,pop_2,T_Mann,T_Frau,Etr_Total,Accidents dégâts matériels,Accidents avec dommages corporels,Morts,GEM_FLAECH,...,canton_2_16.0,canton_2_17.0,canton_2_18.0,canton_2_20.0,canton_2_21.0,canton_2_22.0,canton_2_23.0,canton_2_24.0,canton_2_25.0,canton_2_26.0
0,137.799059,8.319474,6.728629,7.601402,7.650645,5.950643,2.639058,2.079443,0.0,6.381816,...,False,False,False,False,False,False,False,False,False,False
1,121.479057,9.008469,6.542472,8.341649,8.314342,7.058758,4.804021,3.496508,0.0,7.123673,...,False,False,False,True,False,False,False,False,False,False
2,105.489526,7.271009,6.917706,6.573680,6.598509,4.820282,0.000010,-11.512925,0.0,5.587249,...,False,False,False,False,False,False,False,False,False,True
3,108.055025,5.587249,6.359574,5.023881,5.010635,2.890372,0.693152,-11.512925,0.0,5.937536,...,False,False,False,False,False,False,False,False,False,False
4,116.318767,5.384495,6.196444,4.634729,4.753590,2.639058,-11.512925,0.000010,0.0,6.666957,...,False,False,False,False,False,False,False,False,False,False


In [22]:
print(np.unique(y_train, return_counts=True))
print(np.unique(y_val, return_counts=True))
print(np.unique(y_test, return_counts=True))

(array([0, 1]), array([7132444,   95182]))
(array([0, 1]), array([1664686,   20354]))
(array([0, 1]), array([695833,  10023]))


In [23]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

(7227626, 83)
(705856, 87)
(1685040, 85)


In [24]:
X_val  = X_val.reindex(columns=X_train.columns, fill_value=0)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

print(X_train.shape, X_val.shape, X_test.shape)

assert X_train.shape[1] == X_val.shape[1] == X_test.shape[1]
assert X_train.columns.equals(X_val.columns)
assert X_train.columns.equals(X_test.columns)

(7227626, 83) (1685040, 83) (705856, 83)


## XGBoost Classifier

### Initial models

In [15]:
# We start with a basic model 

clf = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    n_estimators=200,
    max_depth=5,
    learning_rate=0.1,
    random_state=42
)

clf.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    verbose=True
)

y_val_proba = clf.predict_proba(X_val)[:, 1]
y_val_pred = (y_val_proba >= 0.5).astype(int)

print(classification_report(y_val, y_val_pred))
print(confusion_matrix(y_val, y_val_pred))


[0]	validation_0-auc:0.99136
[1]	validation_0-auc:0.99236
[2]	validation_0-auc:0.99250
[3]	validation_0-auc:0.99253
[4]	validation_0-auc:0.99260
[5]	validation_0-auc:0.99267
[6]	validation_0-auc:0.99286
[7]	validation_0-auc:0.99292
[8]	validation_0-auc:0.99296
[9]	validation_0-auc:0.99300
[10]	validation_0-auc:0.99306
[11]	validation_0-auc:0.99309
[12]	validation_0-auc:0.99311
[13]	validation_0-auc:0.99312
[14]	validation_0-auc:0.99316
[15]	validation_0-auc:0.99317
[16]	validation_0-auc:0.99319
[17]	validation_0-auc:0.99318
[18]	validation_0-auc:0.99317
[19]	validation_0-auc:0.99318
[20]	validation_0-auc:0.99321
[21]	validation_0-auc:0.99322
[22]	validation_0-auc:0.99323
[23]	validation_0-auc:0.99325
[24]	validation_0-auc:0.99328
[25]	validation_0-auc:0.99328
[26]	validation_0-auc:0.99333
[27]	validation_0-auc:0.99335
[28]	validation_0-auc:0.99339
[29]	validation_0-auc:0.99340
[30]	validation_0-auc:0.99341
[31]	validation_0-auc:0.99343
[32]	validation_0-auc:0.99345
[33]	validation_0-au

In [16]:
n_neg = (y_train == 0).sum()
n_pos = (y_train == 1).sum()

print("Negatives:", n_neg)
print("Positives:", n_pos)
print("Ratio (neg / pos):", n_neg / n_pos)


Negatives: 14264892
Positives: 190360
Ratio (neg / pos): 74.93639420046229


In [17]:
# we add scale_pos_weight to handle class imbalance 

scale_pos_weight = n_neg / n_pos

clf_weighted = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    n_estimators=200,
    max_depth=5,
    learning_rate=0.1,
    scale_pos_weight=scale_pos_weight,
    random_state=42
)

clf_weighted.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    verbose=True
)

y_val_proba_w = clf_weighted.predict_proba(X_val)[:, 1]
y_val_pred_w = (y_val_proba_w >= 0.5).astype(int)

print(classification_report(y_val, y_val_pred_w))
print(confusion_matrix(y_val, y_val_pred_w))



[0]	validation_0-auc:0.98977
[1]	validation_0-auc:0.99035
[2]	validation_0-auc:0.99100
[3]	validation_0-auc:0.99189
[4]	validation_0-auc:0.99194
[5]	validation_0-auc:0.99234
[6]	validation_0-auc:0.99246
[7]	validation_0-auc:0.99248
[8]	validation_0-auc:0.99250
[9]	validation_0-auc:0.99263
[10]	validation_0-auc:0.99268
[11]	validation_0-auc:0.99267
[12]	validation_0-auc:0.99272
[13]	validation_0-auc:0.99289
[14]	validation_0-auc:0.99287
[15]	validation_0-auc:0.99296
[16]	validation_0-auc:0.99299
[17]	validation_0-auc:0.99311
[18]	validation_0-auc:0.99325
[19]	validation_0-auc:0.99323
[20]	validation_0-auc:0.99331
[21]	validation_0-auc:0.99336
[22]	validation_0-auc:0.99340
[23]	validation_0-auc:0.99347
[24]	validation_0-auc:0.99348
[25]	validation_0-auc:0.99357
[26]	validation_0-auc:0.99363
[27]	validation_0-auc:0.99362
[28]	validation_0-auc:0.99363
[29]	validation_0-auc:0.99366
[30]	validation_0-auc:0.99371
[31]	validation_0-auc:0.99372
[32]	validation_0-auc:0.99373
[33]	validation_0-au

In [18]:
# we tune threshold for baseline 
thresholds = np.linspace(0.01, 0.99, 200)
f1_scores = []

for t in thresholds:
    preds = (y_val_proba >= t).astype(int)
    f1_scores.append(f1_score(y_val, preds))

best_t_baseline = thresholds[np.argmax(f1_scores)]
best_f1_baseline = max(f1_scores)

print("Best threshold (baseline):", best_t_baseline)
print("Best F1 (baseline):", best_f1_baseline)


Best threshold (baseline): 0.5369346733668342
Best F1 (baseline): 0.6884701501924577


In [19]:
f1_scores_w = []

for t in thresholds:
    preds = (y_val_proba_w >= t).astype(int)
    f1_scores_w.append(f1_score(y_val, preds))

best_t_weighted = thresholds[np.argmax(f1_scores_w)]
best_f1_weighted = max(f1_scores_w)

print("Best threshold (weighted):", best_t_weighted)
print("Best F1 (weighted):", best_f1_weighted)


Best threshold (weighted): 0.9801507537688443
Best F1 (weighted): 0.7008716786240097


In [20]:
best_threshold = best_t_weighted
final_clf = clf_weighted

In [21]:
y_test_proba = final_clf.predict_proba(X_test)[:, 1]
y_test_pred = (y_test_proba >= best_threshold).astype(int)

print(classification_report(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1391669
           1       0.73      0.69      0.71     20043

    accuracy                           0.99   1411712
   macro avg       0.86      0.84      0.85   1411712
weighted avg       0.99      0.99      0.99   1411712

[[1386495    5174]
 [   6303   13740]]


### Grid search

In [22]:
n_neg = np.sum(y_train == 0)
n_pos = np.sum(y_train == 1)

scale_pos_weight = n_neg / n_pos
print(f"scale_pos_weight = {scale_pos_weight:.2f}")

scale_pos_weight = 74.94


In [23]:
print(X_train.shape)

(14455252, 83)


In [24]:
# since it takes a long time and much memory we do the grid search on a portion of the train data 

X_tune, _, y_tune, _ = train_test_split(
    X_train,
    y_train,
    train_size=0.5,
    stratify=y_train,
    random_state=37
)

print("Tuning set shape:", X_tune.shape)
print("Positive ratio (tune):", y_tune.mean())
print("Positive ratio (train):", y_train.mean())

Tuning set shape: (7227626, 83)
Positive ratio (tune): 0.013168916045185514
Positive ratio (train): 0.013168916045185514


In [25]:
param_distributions = {
    'learning_rate': [0.03, 0.05, 0.1],
    'max_depth': [4, 5, 6, 7],
    'n_estimators': [200, 300],
    'subsample': [0.8, 0.9],
    'colsample_bytree': [0.8, 0.9],
    'min_child_weight': [3, 5],
    'gamma': [0, 0.1, 0.2],
}

base_clf = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    scale_pos_weight=scale_pos_weight,
    tree_method='hist',
    random_state=37,
    n_jobs=-1
)


In [26]:
# Two-phase hyperparameter tuning approach : 

# phase 1: COARSE RANDOM SEARCH


# we start with random search to explore
random_search = RandomizedSearchCV(
    estimator=base_clf,
    param_distributions=param_distributions,
    n_iter=20,
    scoring='roc_auc',
    cv=3,
    verbose=1,
    n_jobs=1,          # before was -1 but changed it to avoid memory explosion
    refit=True,
    random_state=37
)
print("Phase 1: Random Search on tune set")
random_search.fit(X_tune, y_tune)



Phase 1: Random Search on tune set
Fitting 3 folds for each of 20 candidates, totalling 60 fits


,estimator,"XGBClassifier...ree=None, ...)"
,param_distributions,"{'colsample_bytree': [0.8, 0.9], 'gamma': [0, 0.1, ...], 'learning_rate': [0.03, 0.05, ...], 'max_depth': [4, 5, ...], ...}"
,n_iter,20
,scoring,'roc_auc'
,n_jobs,1
,refit,True
,cv,3
,verbose,1
,pre_dispatch,'2*n_jobs'
,random_state,37
,error_score,nan


In [27]:
best_clf = random_search.best_estimator_

y_val_proba = best_clf.predict_proba(X_val)[:, 1]
val_auc = roc_auc_score(y_val, y_val_proba)

print(f"Validation AUC: {val_auc:.4f}")
print("Best parameters:")
for k, v in random_search.best_params_.items():
    print(f"  {k}: {v}")

Validation AUC: 0.9948
Best parameters:
  subsample: 0.8
  n_estimators: 300
  min_child_weight: 5
  max_depth: 6
  learning_rate: 0.1
  gamma: 0.2
  colsample_bytree: 0.9


In [28]:
best_params = random_search.best_params_

fine_grid = {}

for param, value in best_params.items():
    if param in ['learning_rate', 'subsample', 'colsample_bytree']:
        fine_grid[param] = sorted({
            max(0.01, value - 0.02),
            value,
            min(1.0, value + 0.02)
        })
    elif param in ['max_depth', 'min_child_weight']:
        fine_grid[param] = sorted({
            max(1, value - 1),
            value,
            value + 1
        })
    elif param == 'n_estimators':
        fine_grid[param] = sorted({
            max(50, value - 50),
            value,
            value + 50
        })
    elif param == 'gamma':
        fine_grid[param] = sorted({
            max(0, value - 0.1),
            value,
            value + 0.1
        })
    else:
        fine_grid[param] = [value]

param_combinations = list(
    dict(zip(fine_grid.keys(), v))
    for v in product(*fine_grid.values())
)

print(f"Phase 2: testing {len(param_combinations)} combinations")


Phase 2: testing 2187 combinations


In [29]:
best_auc = -np.inf
best_params_final = None
best_model = None

MAX_COMBOS = min(20, len(param_combinations))

for i, params in enumerate(param_combinations[:MAX_COMBOS]):
    print(f"[{i+1}/{MAX_COMBOS}] Testing params...")

    model = xgb.XGBClassifier(
        **params,
        objective='binary:logistic',
        eval_metric='auc',
        scale_pos_weight=scale_pos_weight,
        tree_method='hist',
        random_state=37,
        n_jobs=-1
    )

    # for the fine grid we use the whole train data to be as precise as possible 
    model.fit(X_train, y_train)

    y_val_proba = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, y_val_proba)

    print(f" AUC: {auc:.5f}")

    if auc > best_auc:
        best_auc = auc
        best_params_final = params
        best_model = model
        print("New best model found")

print("PHASE 2 RESULT:")
print(f"Best validation AUC: {best_auc:.5f}")
print("Best params:")
for k, v in best_params_final.items():
    print(f"  {k}: {v}")


[1/20] Testing params...
 AUC: 0.99482
New best model found
[2/20] Testing params...
 AUC: 0.99478
[3/20] Testing params...
 AUC: 0.99478
[4/20] Testing params...
 AUC: 0.99482
New best model found
[5/20] Testing params...
 AUC: 0.99480
[6/20] Testing params...
 AUC: 0.99481
[7/20] Testing params...
 AUC: 0.99486
New best model found
[8/20] Testing params...
 AUC: 0.99480
[9/20] Testing params...
 AUC: 0.99481
[10/20] Testing params...
 AUC: 0.99481
[11/20] Testing params...
 AUC: 0.99488
New best model found
[12/20] Testing params...
 AUC: 0.99484
[13/20] Testing params...
 AUC: 0.99481
[14/20] Testing params...
 AUC: 0.99487
[15/20] Testing params...
 AUC: 0.99484
[16/20] Testing params...
 AUC: 0.99481
[17/20] Testing params...
 AUC: 0.99487
[18/20] Testing params...
 AUC: 0.99484
[19/20] Testing params...
 AUC: 0.99486
[20/20] Testing params...
 AUC: 0.99491
New best model found
PHASE 2 RESULT:
Best validation AUC: 0.99491
Best params:
  subsample: 0.78
  n_estimators: 250
  min_ch

In [30]:
y_val_proba = best_model.predict_proba(X_val)[:, 1]

thresholds = np.linspace(0.05, 0.95, 150)
f1_scores = []

for t in thresholds:
    y_pred = (y_val_proba >= t).astype(int)
    f1_scores.append(f1_score(y_val, y_pred))

best_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_idx]
best_f1 = f1_scores[best_idx]

print(f"Best threshold: {best_threshold:.4f}")
print(f"Best validation F1: {best_f1:.4f}")


Best threshold: 0.9500
Best validation F1: 0.6430


In [ ]:
with open("xgboost_best_clf_params.json", "w") as f:
    json.dump(best_params_final, f, indent=2)


In [31]:
best_model.save_model("xgboost_best_clf_model.json")

### Final retrain and evaluation

In [32]:
# we combine train + validation to retrain with the hyperparameters we found
X_full = pd.concat([X_train, X_val])
y_full = pd.concat([y_train, y_val])

final_clf = xgb.XGBClassifier(
    **best_params_final,
    objective='binary:logistic',
    eval_metric='auc',
    scale_pos_weight=scale_pos_weight,
    tree_method='hist',
    random_state=37,
    n_jobs=-1
)

final_clf.fit(X_full, y_full)


,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.9
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'auc'


In [33]:
# Predict test probabilities
y_test_proba = final_clf.predict_proba(X_test)[:, 1]
y_test_pred = (y_test_proba >= best_threshold).astype(int)

print("TEST AUC:", roc_auc_score(y_test, y_test_proba))
print(classification_report(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))
print(f"Final F1 score : {f1_score(y_test, y_test_pred)}")

TEST AUC: 0.9939364933562427
              precision    recall  f1-score   support

           0       1.00      0.99      0.99   1391669
           1       0.59      0.82      0.68     20043

    accuracy                           0.99   1411712
   macro avg       0.79      0.90      0.84   1411712
weighted avg       0.99      0.99      0.99   1411712

[[1380075   11594]
 [   3649   16394]]
Final F1 score : 0.6826424600778663


## XGBoost Regressor

### Reprepare data for regression

In [25]:
drop_features = ['commune_1', 'commune_2', 'flow']

X_train_reg, y_train_reg, X_val_reg, y_val_reg, X_test_reg, y_test_reg = build_train_test_val(df, 
                                                                      test_canton_ids=[3], 
                                                                      val_canton_ids=[19], 
                                                                      zero_drop_ratio=1, 
                                                                      random_state=37, 
                                                                      features=drop_features, 
                                                                      classify= False)


Splitting with canton [3] as test set:
Total flows: 125,559
Train size: 115,536 rows (92.0%)
Test size: 10,023 rows (8.0%)
Splitting with canton [19] as test set:
Total flows: 115,536
Train size: 95,182 rows (82.4%)
Test size: 20,354 rows (17.6%)
Dropping 100% of zeros in training data


In [26]:
X_train_reg = pd.get_dummies(
    X_train_reg,
    columns=["canton_code", "canton_code_2"],
    prefix=["canton_1", "canton_2"],
    drop_first=False
)

X_test_reg = pd.get_dummies(
    X_test_reg,
    columns=["canton_code", "canton_code_2"],
    prefix=["canton_1", "canton_2"],
    drop_first=False
)

X_val_reg = pd.get_dummies(
    X_val_reg,
    columns=["canton_code", "canton_code_2"],
    prefix=["canton_1", "canton_2"],
    drop_first=False
)

print(X_train_reg.shape)
print(X_test_reg.shape)
print(X_val_reg.shape)

(95182, 83)
(10023, 80)
(20354, 71)


In [27]:
X_val_reg  = X_val_reg.reindex(columns=X_train_reg.columns, fill_value=0)
X_test_reg = X_test_reg.reindex(columns=X_train_reg.columns, fill_value=0)

print(X_train_reg.shape, X_val_reg.shape, X_test_reg.shape)

assert X_train_reg.shape[1] == X_val_reg.shape[1] == X_test_reg.shape[1]
assert X_train_reg.columns.equals(X_val_reg.columns)
assert X_train_reg.columns.equals(X_test_reg.columns)

(95182, 83) (20354, 83) (10023, 83)


In [28]:
print(X_train_reg.shape, X_val_reg.shape, X_test_reg.shape)
print(y_train_reg.describe())


(95182, 83) (20354, 83) (10023, 83)
count    95182.000000
mean        43.375659
std        176.994846
min          5.000000
25%          7.000000
50%         12.000000
75%         28.000000
max      11129.000000
Name: flow, dtype: float64


In [29]:
y_train_reg_log = np.log1p(y_train_reg)
y_val_reg_log   = np.log1p(y_val_reg)
y_test_reg_log  = np.log1p(y_test_reg)


In [30]:
reg = xgb.XGBRegressor(
    objective="reg:squarederror",
    n_estimators=500,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
)

reg.fit(
    X_train_reg,
    y_train_reg_log,
    eval_set=[(X_val_reg, y_val_reg_log)],
    verbose=True
)

y_val_pred_log = reg.predict(X_val_reg)
y_val_pred = np.expm1(y_val_pred_log)

y_val_true = y_val_reg

mae  = mean_absolute_error(y_val_true, y_val_pred)
mse = mean_squared_error(y_val_true, y_val_pred)
rmse = np.sqrt(mse)

print("MAE:", mae)
print("RMSE:", rmse)


[0]	validation_0-rmse:0.87937
[1]	validation_0-rmse:0.85821
[2]	validation_0-rmse:0.84137
[3]	validation_0-rmse:0.82497
[4]	validation_0-rmse:0.80760
[5]	validation_0-rmse:0.79108
[6]	validation_0-rmse:0.77589
[7]	validation_0-rmse:0.76108
[8]	validation_0-rmse:0.74756
[9]	validation_0-rmse:0.73569
[10]	validation_0-rmse:0.72407
[11]	validation_0-rmse:0.71329
[12]	validation_0-rmse:0.70291
[13]	validation_0-rmse:0.69452
[14]	validation_0-rmse:0.68603
[15]	validation_0-rmse:0.67834
[16]	validation_0-rmse:0.67004
[17]	validation_0-rmse:0.66306
[18]	validation_0-rmse:0.65645
[19]	validation_0-rmse:0.65022
[20]	validation_0-rmse:0.64427
[21]	validation_0-rmse:0.64101
[22]	validation_0-rmse:0.63531
[23]	validation_0-rmse:0.63094
[24]	validation_0-rmse:0.62652
[25]	validation_0-rmse:0.62187
[26]	validation_0-rmse:0.61755
[27]	validation_0-rmse:0.61476
[28]	validation_0-rmse:0.61110
[29]	validation_0-rmse:0.60815
[30]	validation_0-rmse:0.60489
[31]	validation_0-rmse:0.60170
[32]	validation_0-

### Grid search

In [31]:
# We do again a grid search in 2 phases 

param_grid_reg = {
    "n_estimators": [200, 400, 600],
    "max_depth": [4, 6, 8],
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample": [0.7, 0.8, 0.9],
    "colsample_bytree": [0.7, 0.8, 0.9],
    "min_child_weight": [1, 3, 5],
    "gamma": [0, 0.1, 0.3]
}


reg_base = xgb.XGBRegressor(
    objective="reg:squarederror",
    random_state=37,
    n_jobs=1
)

random_search_reg = RandomizedSearchCV(
    estimator=reg_base,
    param_distributions=param_grid_reg,
    n_iter=25,
    scoring="neg_root_mean_squared_error",  # RMSE on log-scale
    cv=3,
    refit=True,
    random_state=37,
    n_jobs=1,
    verbose=1
)

random_search_reg.fit(X_train_reg, y_train_reg_log)

print("Best CV RMSE (log):", -random_search_reg.best_score_)
print("Best params:")
for k, v in random_search_reg.best_params_.items():
    print(f"  {k}: {v}")


Fitting 3 folds for each of 25 candidates, totalling 75 fits
Best CV RMSE (log): 0.45553308150695043
Best params:
  subsample: 0.9
  n_estimators: 600
  min_child_weight: 3
  max_depth: 6
  learning_rate: 0.1
  gamma: 0.3
  colsample_bytree: 0.8


In [32]:
# validation for phase 1

best_reg_phase1 = random_search_reg.best_estimator_

y_val_pred_log = best_reg_phase1.predict(X_val_reg)
y_val_pred = np.expm1(y_val_pred_log)

mae = mean_absolute_error(y_val_reg, y_val_pred)
rmse = np.sqrt(mean_squared_error(y_val_reg, y_val_pred))

print("Phase 1 MAE:", mae)
print("Phase 1 RMSE:", rmse)


Phase 1 MAE: 12.883836392594002
Phase 1 RMSE: 39.80515398937253


In [33]:
# phase 2: we tune around the best parameters we found in phase 1

best_params = random_search_reg.best_params_

fine_grid = {}

for param, value in best_params.items():
    if param in ["learning_rate", "subsample", "colsample_bytree"]:
        fine_grid[param] = [max(0.01, value - 0.02), value, min(1.0, value + 0.02)]
    elif param in ["max_depth", "min_child_weight"]:
        fine_grid[param] = [max(1, value - 1), value, value + 1]
    elif param == "n_estimators":
        fine_grid[param] = [max(100, value - 100), value, value + 100]
    elif param == "gamma":
        fine_grid[param] = [max(0, value - 0.1), value, value + 0.1]
    else:
        fine_grid[param] = [value]

In [34]:
best_rmse = float("inf")
best_reg_final = None
best_params_final = None

param_combinations = [
    dict(zip(fine_grid.keys(), v))
    for v in product(*fine_grid.values())
]

for i, params in enumerate(param_combinations[:25]):
    print(f"Testing combo {i+1}/{min(20, len(param_combinations))}")

    reg = xgb.XGBRegressor(
        objective="reg:squarederror",
        random_state=37,
        n_jobs=-1,
        **params
    )

    reg.fit(X_train_reg, y_train_reg_log)
    y_pred_log = reg.predict(X_val_reg)
    y_pred = np.expm1(y_pred_log)

    rmse = np.sqrt(mean_squared_error(y_val_reg, y_pred))

    if rmse < best_rmse:
        best_rmse = rmse
        best_reg_final = reg
        best_params_final = params
        print(f"  New best RMSE: {rmse:.2f}")


Testing combo 1/20
  New best RMSE: 38.72
Testing combo 2/20
  New best RMSE: 38.01
Testing combo 3/20
Testing combo 4/20
Testing combo 5/20
Testing combo 6/20
Testing combo 7/20
  New best RMSE: 37.92
Testing combo 8/20
Testing combo 9/20
Testing combo 10/20
Testing combo 11/20
Testing combo 12/20
Testing combo 13/20
Testing combo 14/20
Testing combo 15/20
Testing combo 16/20
Testing combo 17/20
Testing combo 18/20
Testing combo 19/20
Testing combo 20/20
Testing combo 21/20
Testing combo 22/20
Testing combo 23/20
Testing combo 24/20
Testing combo 25/20


In [ ]:
with open("xgboost_best_reg_params.json", "w") as f:
    json.dump(best_params_final, f, indent=2)


In [36]:
best_reg_final.save_model("xgboost_best_reg_model.json")

### Final retrain

In [45]:
X_full_reg = pd.concat([X_train_reg, X_val_reg])
y_full_reg = pd.concat([y_train_reg, y_val_reg])

best_reg_final.fit(
    X_full_reg,
    np.log1p(y_full_reg)
)

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.78
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [46]:
predicred_reg_log = best_reg_final.predict(X_test_reg)
predicted_reg = np.expm1(predicred_reg_log)

print(y_test_reg.shape)
print(predicted_reg.shape)

r2_reg = r2_score(y_test_reg, predicted_reg)
rmse_reg = np.sqrt(mean_squared_error(y_test_reg, predicted_reg))
mae_reg = np.mean(np.abs(y_test_reg - predicted_reg))
correlation_reg = np.corrcoef(y_test_reg, predicted_reg)[0, 1]
    
print(f"R² Score: {r2_reg:.3f}")
print(f"RMSE: {rmse_reg:.1f} commuters")
print(f"MAE: {mae_reg:.1f} commuters")
print(f"Pearson Correlation: {correlation_reg:.3f}")

(20043,)
(20043,)
R² Score: 0.776
RMSE: 57.9 commuters
MAE: 16.5 commuters
Pearson Correlation: 0.909
